In [1]:
from pathlib import Path


import ipywidgets as widgets
import fastai.vision.all as vision
import requests

In [2]:
from download_data import download_file_from_google_drive, DEFAULT_MODEL_LOCATION

In [3]:
if not DEFAULT_MODEL_LOCATION.exists():
    download_file_from_google_drive()

In [4]:
model = vision.load_learner(DEFAULT_MODEL_LOCATION)

In [5]:
url = 'https://static.toiimg.com/photo/msid-67586673/67586673.jpg?3918697'
img = vision.PILImage.create(requests.get(url, stream=True).raw)


In [6]:
button = widgets.Button(
    description='Start',
    disabled=False,
    button_style='info', # 'success', 'info', 'warning', 'danger' or ''
    tooltip='Start',
    icon='play' # (FontAwesome names without the `fa-` prefix)
)

def on_click_classify(change):
    img = vision.PILImage.create(requests.get(url, stream=True).raw)
    out_l.clear_output(wait=True)
    out_r.clear_output(wait=True)

    pred,pred_idx,probs = model.predict(img)
    prob = probs[pred_idx] * 100
    s = f"I'm {prob:.0f}% sure that is {pred.title()}"
    pred_image = pred.replace(' ', '_') + '.jpg'
    p = vision.PILImage.create(Path(pred_image).read_bytes())
    with out_l:
        display(img.to_thumb(500,500))
    with out_r:
        display(p.to_thumb(500, 500))
    lab.value = s

button.on_click(on_click_classify)

In [7]:
user_url = widgets.Text(
    value='https://',
    placeholder='Type something',
    description='',
    disabled=False,
    layout=widgets.Layout(width='80%')
    
)

out_l = widgets.Output()
out_r = widgets.Output()
lab = widgets.Label()

url_row = widgets.HBox([user_url], layout=widgets.Layout(justify_content='center', margin='5px'))
button_row = widgets.HBox([button], layout=widgets.Layout(justify_content='center', margin='5px'))
bottom_box = widgets.HBox([out_l, out_r], layout=widgets.Layout(width='100%', justify_content='space-around', display='flex-box', align_items='center'))
bb = widgets.HBox([lab])

widgets.VBox([url_row, button_row, bottom_box, bb])